# Tutorial 1: Data Preprocessing

---

### Introduction

Welcome! This tutorial will show you how to visualise and preprocess astronomical data using python. From this tutorial, you will learn the following:

1. How to download astronomical data
2. How to read and visualise data
3. How to preprocess imagery data

The data [GalaxyMNIST](https://github.com/mwalmsley/galaxy_mnist) contains 10,000 images of galaxies (either 3x64x64 or 3x224x224), confidently labelled by Galaxy Zoo volunteers as belonging to one of four morphology classes, where the classes are:

1. smooth and round
2. smooth and cigar-shaped
3. edge-on-disk
4. unbarred spiral

The galaxies are selected from `Galaxy Zoo DECaLS Campaign` A (GZD-A). The images are shown to volunteers on Galaxy Zoo for them to classify. At least 17 people must have been asked the necessary questions, and at least half of them must have answered with the given class. The class labels are, therefore, much more confident than, for example, simply labelling with the most common answer to some question. For more info, visit this [link](https://github.com/mwalmsley/galaxy_mnist)


Labelling data is a big topic in machine learning. Labelled data allow us to use supervised machine learning tools to train on labelled data and classify unlabeled ones. However, labelling data can be costly. Scientists have been trying to figure out a way to label data, but the easiest is that we can depend on human knowledge to label data like the `Galaxy Zoo DECaLS Campaign`.  However, unlabeled data can also be of some benefit if analyzed with the right tools, like unsupervised machine learning. Those tools allow us to explore (find patterns) and cluster the data, leading to discoveries in the data.


---

### Downloading data

First, let us call the python script that will let us download the data.

In [ ]:
from galaxy_mnist import GalaxyMNISTHighrez

'GalaxyMNISTHighrez' class has the follwing attribuits: 
 - `root`: Specifing the dir to download the data
 - `download:` A boolean value, `True` to download the images
 - `train:` A boolean value, `True` to assign the just the training data, `False` will assgin just the testing data. 
   - `Notice:` The data has a fixed 80/20 train/test division.

In [ ]:
# for the training data
dataset_train = GalaxyMNISTHighrez(
    root='data_import/data',
    download=True,
    train=True  # by default True, or False for canonical test set
)
# for the testing data
dataset_test = GalaxyMNISTHighrez(
    root='data_import/data',
    download=True,
    train=False  # by default True, or False for canonical test set
)

---

### Reading data

first, let us read inputs `images` and outputs `labels` into two seperate variables

In [ ]:
images_train = dataset_train.data
labels_train = dataset_train.targets

let us check the shape of the two datasets (traning, and testing)

In [ ]:
print("Shape of the input (training) "+str(images_train.shape)+ ", type: "+ str(images_train.dtype))
print("Shape of the output (training) "+str(labels_train.shape)+ ", type: "+ str(labels_train.dtype))



we notice the following:
 1. The input data has 8000 samples with 3 channels, which stands for `rgb` colors, where each sample has a size of 224
 2. the output is just one dimnsional array, which contains the lables of the samples, and as we mentioned above the label can be one of the following: a)  2. the output is just one dimnsional array, which contains the lables of the samples.

**Exercise 1:**
Store the testing set inputs and outputs into two different variables, and display their shape.

In [ ]:
### -- Code here --



Now, let us check the frequency of the labels/output, but fiest let us orint them out.

In [ ]:
GalaxyMNISTHighrez.classes

   **Where**:
   - 0: `smooth_round`,
   - 1: `smooth_cigar`,
   - 2: `edge_on_disk`,
   - 3: `unbarred_spiral`

to count the frequncy of the labels, we will convert the data into pandas dataframe
@@ Ezra, do you think we can do this step better?

In [ ]:
import pandas as pd # for importing data into data frame format

In [ ]:
df_labelsTrain = pd.DataFrame(labels_train)
dict_train =  df_labelsTrain.stack().value_counts().to_dict()
print(dict_train)

### Data visualisation

we can also try to visualse label frequencies in a bar graph, but first let us call the package that will allow us to draw the graphs

In [ ]:
import matplotlib.pyplot as plt #This displays graphs once they have been created

In [ ]:


for newKey, oldKey in  zip( GalaxyMNISTHighrez.classes, sorted(dict_train.keys()) ): 
    dict_train[newKey] = dict_train.pop(oldKey)  # changing the key names


plt.bar(range(len(dict_train)), list(dict_train.values()), align='center')
plt.xticks(range(len(dict_train)), list(dict_train.keys()))
print("Dict: "+str(dict_train))

plt.show()

**Exercise 2:**
Count the frequency for the testing data and visualise it

In [ ]:
### -- Code here --


Now let us take a look at the images of the different galaxy shapes

In [ ]:
import numpy as np # For handling N-DIMENSIONAL ARRAYS
from PIL import Image #  PIL is the Python Imaging Library

In [ ]:
class0 = np.where (np.array(labels_train) == 0 )
class1 = np.where (np.array(labels_train) == 1 )
class2 = np.where (np.array(labels_train) == 2 )
class3 = np.where (np.array(labels_train) == 3 )

In [ ]:
rows = 1
columns = 5
for classArr in [class0, class1, class2, class3]:
    fig = plt.figure(figsize=(8, 8))
    for i in range (5):
        train_image, train_label = dataset_train[classArr[0][i]]
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(train_image)
    print("label: "+str(GalaxyMNISTHighrez.classes[train_label]))
    plt.tight_layout()
    plt.show()     # increase the number of images per class

The resolution of this dataset does not seem to be good, `smooth cigar` and `edge_on_disk` kind of look like each other, which will give us problems later on when performing classification or clustering.

---

### Data preprocessing

Classical machine learning usually requires the data to go under many layers of processing so it can be fit for training (e.g. preprocessing, feature extraction, optimization, ..). For each layer, many different algorithms may be used. 

On the other hand, deep learning models with enough data are often referred to as End-to-End learning models ([E2E](https://towardsdatascience.com/e2e-the-every-purpose-ml-method-5d4f20dafee4)), since it can extract information from raw data.

In the following tutorial, we will use an autoencoder (deep learning) model to extract features. Therefore we can feed it the raw data. So a question arises, why are we preprocessing the data? Well, for two reasons related to time complexity. We need to train Deep learning models in a reasonable time so that we can bring the idea across, but more importantly, tuning the hyperparameters for deep learning models can be very tough; therefore, if we think that we can manually reduce the complexity of the data without losing lots/any information that would be a perfect idea. 

But How Can we tell if we lost lots of info? In our case, since the data is imagery data, we can notify in two ways.
a) if our eyes can manually still classify the images, we can conclude that the preprocessing is a good idea.
b) testing the results of the preprocessed data on a validation set and comparing it against the original/raw data.

This is a suggested pipeline. However, participants are encouraged to explore.

In this tutorial, we will try to reduce the complexity of the data by doing the following:
   - Reduce the number of channels from RGB to greyscale.
   - Reduce the size of the image by a factor of 4.
   - Perform normalization (divide by 255)

#### a- [Grayscale](https://www.tutorialspoint.com/pytorch-how-to-convert-an-image-to-grayscale)

In [ ]:
import torchvision.transforms as transforms #Transforms are for common image transformations

In [ ]:
images_train.shape

We can see that the original image size has three channels which stand for the RGB colours, now let us perfom the `greyscalling`

In [ ]:
transformGrey = transforms.Grayscale()
images_trainGrey = transformGrey(images_train)

Now let us check the size of the tensor

In [ ]:
images_trainGrey.shape

we can see the number of channels went down from 3 to 1

**Exercise 3:**
Visualise the grayscale images for all classes

In [ ]:
### -- Code here__

rows = 1
columns = 5

for classArr in [ [class0,0], [class1,1], [class2,2], [class3,3]]:
    fig = plt.figure(figsize=(10, 10))
    for i in range (5):
        train_image = images_trainGrey[classArr[0][0][i]]
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(train_image[0],cmap='gray', vmin=0, vmax=255 )
    print("label: "+str(GalaxyMNISTHighrez.classes[classArr[1]]))
    plt.tight_layout()
    plt.show()  
    

We can notice that visually that nothing changed much from the original data. However, we reduced the complexity of the data by a favour of 3, which is enormous!

#### b- [Size reduction](https://www.tutorialspoint.com/pytorch-how-to-resize-an-image-to-a-given-size)

In [ ]:
transformResize = transforms.Resize(56)

Now let us reduce the image, we will reduce it on the grayscale images

In [ ]:
images_trainResized = transformResize(images_trainGrey)

now let us check the size

In [ ]:
images_trainResized.shape

**Exercise 4:**
Visualise the resized images for all classes

In [ ]:
#___ Code here___

for classArr in [ [class0,0], [class1,1], [class2,2], [class3,3]]:
    fig = plt.figure(figsize=(10, 10))
    for i in range (5):
        train_image = images_trainResized[classArr[0][0][i]]
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(train_image[0],cmap='gray', vmin=0, vmax=255 )
    print("label: "+str(GalaxyMNISTHighrez.classes[classArr[1]]))
    plt.tight_layout()
    plt.show()  

#### c- Normalization

Now we will simply try to devide the pixel values by 255. Where the goal is to transform features to be on a similar scale ([source](https://developers.google.com/machine-learning/data-prep/transform/normalization)).

In [ ]:
images_trainPre = images_trainResized/255

Now let us check the values before and after the nomalisatiom

In [ ]:
print(images_trainResized[0][0][0])
print(images_trainPre[0][0][0])

##### **_End of the pre-processing pipeline_**

Now the above pre-processing is not the only way where we can reduce the features, the following can also be done:
1. Cropping images away from the centre
2. Image segmentation, maybe to remove background noise (Deep learning can be used for this)
3. ?

**Exercise 5**: Perform the pre-processing pipeline but on the testing set

In [ ]:
#___ Code here___


---

### **_End of Tutorial 1_**